In [1]:
### Pull the acquired data from the csv file
import csv
import pandas
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import sklearn
from random import shuffle
import os
import csv

samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)



In [2]:
### Define the generator and prepare the data

In [3]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('\\')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)


In [ ]:
### Extract the training data

In [ ]:
# Using only the center camera images as training data to the model
with open('./driving_log.csv') as csvfile:
    data = pandas.read_csv(csvfile, header=None)

data_array = np.array(data) # convert to numpy array
center_view = list()
right_view = list()
left_view = list()

#cen_images = data_array[:,0]
cen_images = glob.glob('./IMG/center*.jpg')
rig_images = glob.glob('./IMG/right*.jpg')
lef_images = glob.glob('./IMG/left*.jpg')

for i in cen_images:
    center_view.append(cv2.imread(i))
for i in rig_images:
    right_view.append(cv2.imread(i))
for i in lef_images:
    left_view.append(cv2.imread(i))

center_view = np.array(center_view) # image array ready!
right_view = np.array(right_view) # image array ready!
left_view = np.array(left_view) # image array ready!
#print(center_view.shape) 

steer_angle = data_array[:,3] # steer-angle array ready!
steer_angle_left = data_array[:,3]+0.2 # steer-angle array ready!
steer_angle_right = data_array[:,3]-0.2 # steer-angle array ready!
#print(steer_angle.shape)

In [ ]:
### Augment the dataset

In [ ]:
# For a more generalized model, more training data can be created be flipping the images along the vertical axis
center_view_flip = np.flip(center_view, axis=2)
print(('Shape of center_view_flip:{}').format(center_view_flip.shape))
print(('Shape of center_view:{}').format(center_view.shape))

index=1000
# Full frame
plt.figure()
plt.imshow(np.squeeze(center_view[index]))
#print(('Steering angle:{}').format(steer_angle[index]))
plt.title('160x320 frame - steer_angle: ' + str(steer_angle[index]))
plt.figure()
plt.imshow(np.squeeze(center_view_flip[index]))
#print(('Steering angle:{}').format(-steer_angle[index]))
plt.title('160x320 frame - steer_angle: ' + str(-steer_angle[index]))

# Cropped frame
crop_y_top = 70 # Suggested value was 50
crop_y_bot = 20

def crop_array(crop_y_top, crop_y_bot, arr):
    arr_crop = arr[:, crop_y_top:arr.shape[1]-crop_y_bot, :]
    return arr_crop

center_view_crop = crop_array(crop_y_top, crop_y_bot, center_view)
center_view_flip_crop = crop_array(crop_y_top, crop_y_bot, center_view_flip)
right_view_crop = crop_array(crop_y_top, crop_y_bot, right_view)
left_view_crop = crop_array(crop_y_top, crop_y_bot, left_view)

# Display the images
plt.figure()
plt.imshow(np.squeeze(center_view_crop[index]))
plt.title('Cropped ' + str(center_view.shape[1]-crop_y_bot-crop_y_top) + 'x' + str(320) + ' frame - steer_angle: ' + str(steer_angle[index]))

plt.figure()
plt.imshow(np.squeeze(center_view_flip_crop[index]))
plt.title('Cropped ' + str(center_view_flip_crop.shape[1]-crop_y_bot-crop_y_top) + 'x' + str(320) + ' frame - steer_angle: ' + str(-steer_angle[index]))

plt.figure()
plt.imshow(np.squeeze(right_view_crop[index]))
plt.title('Cropped ' + str(right_view_crop.shape[1]-crop_y_bot-crop_y_top) + 'x' + str(320) + ' frame - steer_angle: ' + str(steer_angle_right[index]))

plt.figure()
plt.imshow(np.squeeze(left_view_crop[index]))
plt.title('Cropped ' + str(left_view_crop.shape[1]-crop_y_bot-crop_y_top) + 'x' + str(320) + ' frame - steer_angle: ' + str(steer_angle_left[index]))


# Dimensions after cropping
print(('Shape of center_view_flip_crop:{}').format(center_view_flip_crop.shape))
print(('Shape of center_view_crop:{}').format(center_view_crop.shape))
print(('Shape of center_view:{}').format(center_view.shape))
print(('Shape of right_view:{}').format(right_view.shape))
print(('Shape of left_view:{}').format(left_view.shape))


In [ ]:
### Create the dataset

In [ ]:
# Assign training data
X_train = np.concatenate((center_view, center_view_flip, left_view, right_view))
y_train = np.concatenate((steer_angle, -steer_angle, steer_angle_left, steer_angle_right))
print(('Shape of X_train:{}').format(X_train.shape))
print(('Shape of y_train:{}').format(y_train.shape))
X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
print(('Shape of X_train:{}').format(X_train.shape))
print(('Shape of y_train:{}').format(y_train.shape))

In [ ]:
### Create and run the model

In [ ]:
import keras
from keras.layers import Flatten, Convolution2D, Dense, Activation, Dropout, Lambda, Cropping2D
from keras.models import Sequential

# Modified NVIDIA model
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))) # Normalize and mean center the data
model.add(Cropping2D(cropping=((crop_y_top,crop_y_bot), (0,0))))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
model.add(Dropout(0.50))
model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48,3,3, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3, subsample=(2,2), activation='relu'))
model.add(Convolution2D(72,3,3, subsample=(2,2), activation='relu'))
#model.add(Convolution2D(84,3,3, subsample=(2,2), activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))

model.summary()

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, nb_epoch=5, verbose=1)
#Generator not used because sufficient memory already available
#model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=7, verbose=1)
model.save('model.h5')
print()
print('--- Model saved!')